# Spectral Line Formation

## Numerical implementation

### Line optical depth
Velocity gradients play a key role in line radiative transfer. Since spectral lines are narrowly peaked in frequency space, they are very sensitve to Doppler shifts, and thus motion (gradients), along the line-of-sight.
Therefore, when numerically solving a line transfer problem, it is key to properly trace the velocity (gradient) along the line-of-sight. 
Since we assume to know the line profile function analytically, we can take care of this sharp frequency dependence by resolving its dependenc analytically.

Consider a line-of-sight segment between two consequtive elements, indexed as 0 and 1, parametrized by $\lambda \in [0, 1]$.
The line optical depth in this segment can then be written as,
\begin{equation*}
\chi(\lambda) \ = \ a(\lambda) \, \exp \left(-b(\lambda)^{2}\right) .
\end{equation*}
where we defined,
\begin{align*}
a(\lambda) \ &= \ \frac{\chi_{ij}(\lambda) \, n(\lambda)}{\sqrt{\pi} \, \delta\nu_{ij}(\lambda)}, \\
b(\lambda) \ &= \ \frac{1}{\delta \nu_{ij}(\lambda)} \left\{ \left( 1 + \frac{v_{z}(\lambda)}{c} \right) \nu  - \nu_{ij} \right\} .
\end{align*}
The strongly peaked behaviour is mainly caused by the exponential function. We can resolve this, for instance in the computation of the optical depth, by using linear interpolation functions for $a$ and $b$ while explicitly integrating the exponential.
This yields the optical depth increment,
\begin{equation*}
\Delta \tau \ = \ \Delta x \int_{0}^{1} \text{d}\lambda \ \chi (\lambda) .
\end{equation*}
Using the linear interpolation scheme,
\begin{align*}
a(\lambda) \ &= \ (1-\lambda) a_{0} \ + \ \lambda a_{1}, \\
b(\lambda) \ &= \ (1-\lambda) b_{0} \ + \ \lambda b_{1},
\end{align*}
for the functions $a$ and $b$, this yields,
\begin{equation*}
\Delta \tau \ = \ \frac{\Delta x}{2\left(b_{1}-b_{0}\right)^{2}} \left\{ \left(a_{1}-a_{0}\right) \left( e^{-b_{0}^{2}} - e^{-b_{1}^{2}}\right) + \sqrt{\pi} \left(b_{0}a_{1}-b_{1}a_{0}\right) \big(\text{Erf}(b_{0}) - \text{Erf}(b_{1})\big) \right\} .
\end{equation*}
This expression is numerically stable as long as $b_1$ is not too close to $b_0$, but will suffer from cancelation errors otherwise.

Therefore, for $\left|b_{1}-b_{0}\right| < 10^{-3}$, we use the first four terms of the Taylor expansion of $b_1$ around $b_0$,
\begin{equation*}
\begin{split}
\Delta \tau \ \approx \ \Delta x \, e^{-b_{0}^{2}} & \left(\frac{1}{2}\left(a_{0} + a_{1}\right) \right. \\
&        \ \ \ \ - \ \frac{1}{3} \, \left( a_{0} + 2 a_{1} \right) b_{0} \left(b_{1}-b_{0}\right) \\
&        \ \ \ \ + \ \frac{1}{12} \, \left( a_{0} + 3 a_{1} \right) \left(2 b_{0}^{2} - 1\right) \left(b_{1}-b_{0}\right)^{2} \\
& \left. \ \ \ \ - \ \frac{1}{30} \,  \left( a_{0} + 4 a_{1} \right) \left(2 b_{0}^{2} - 3\right) b_{0} \left(b_{1}-b_{0}\right)^{2} \ + \ \ldots\right) .
\end{split}
\end{equation*}

The implementation on this line optical depth can be found in [src/p3droslo/lines.py](https://github.com/Magritte-code/p3droslo/blob/main/src/p3droslo/lines.py).

### Line radiative transfer

Consider a line-of-sight segment between two consequtive elements, indexed as 0 and 1, parametrized by $\lambda \in [0, 1]$.
Ignoring any incoming radiation, the accumulated intentisty in this segment can then be written as,
\begin{equation*}
\Delta I \ = \ \Delta\tau \int_{0}^{1} \text{d}\lambda \ S(\lambda) \, e^{-\lambda}.
\end{equation*}
where the source function is defined as,
\begin{equation*}
S \ \equiv \ \frac{\eta}{\chi} .
\end{equation*}
Using the linear interpolation scheme,
\begin{align*}
S   (\lambda) \ &= \ (1-\lambda)    S_{0} \ + \ \lambda    S_{1}, \\
\tau(\lambda) \ &= \ (1-\lambda) \tau_{0} \ + \ \lambda \tau_{1},
\end{align*}
for the source function, $S$, and the optical depth, $tau$, this yields,
\begin{equation*}
\Delta I \ = \ S_{0} \, e^{-\tau_{0}} \left( e^{-\Delta\tau} - (1 - \Delta\tau) \right) \ + \
S_{1} \, e^{-\tau_{1}} \left( e^{+\Delta\tau} - (1 + \Delta\tau) \right),
\end{equation*}
where $\Delta\tau \equiv \tau_{1} - \tau_{0}$. This expression is numerically stable as long as $\Delta \tau$ is not too small, but will suffer from cancellation errors otherwise.

Therefore, for $\Delta \tau < 10^{-2} we use the first four terms in the Taylor expansion,
\begin{align*}
e^{-\Delta\tau} - (1 - \Delta\tau) \ &\approx \ \frac{1}{2} \ - \ \frac{1}{6}\Delta\tau \ + \ \frac{1}{120}\Delta\tau^{2} \ - \  \frac{1}{24}\Delta\tau^{3} \ + \ \ldots \\
e^{+\Delta\tau} - (1 + \Delta\tau) \ &\approx \ \frac{1}{2} \ + \ \frac{1}{6}\Delta\tau \ + \ \frac{1}{120}\Delta\tau^{2} \ + \  \frac{1}{24}\Delta\tau^{3} \ + \ \ldots
\end{align*}
The implementation of this intensity increment can be found in [src/p3droslo/forward.py](https://github.com/Magritte-code/p3droslo/blob/main/src/p3droslo/forward.py).